In [45]:
import os
import numpy as np
import random
import pickle
import copy
from matplotlib import pyplot as plt
from datetime import datetime
from itertools import combinations
import h5py

from scipy.stats import norm

np.random.seed(0)
random.seed(0)

In [2]:
new_num_kfolds = 7
NUM_CLIENTS = 14
local_round_threshold = 20
num_usuable_updates = 7

## Mutual Information

In [9]:
# Not used right now. Probably also out of date

# INTRA
'09-19_22-56_NOFL'
'09-19_23-04_FEDAVG' 
'09-19_23-05_PFAFO_GDLS'

# CROSS
'09-19_23-07_PFAFO_GDLS'
'09-19_23-25_FEDAVG'
'09-19_23-30_NOFL'

# This is for the mutual info stuff really...

cv_results_path = r"C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\PythonVersion\PythonSimsRevamp\results\FinalSims"
nofl_filename = "FULLSCIPYMIN_NOFL_KFold0.h5" 
fedavg_filename = "GDLS_FEDAVG_KFold0.h5" 

directory_str = "09-19_23-25_FEDAVG"
h5_path = os.path.join(cv_results_path, directory_str, fedavg_filename)

# This is also not used right now, I don't think?

if False:
    with h5py.File(h5_path, 'r') as f:
        a_group_key = list(f.keys())#[0]
        print(a_group_key)
    
        local_test_error_log = f['local_test_error_log'][()]
        local_train_error_log = f['local_train_error_log'][()]
        
        client_local_model_log_keys = list(f['client_local_model_log'].keys())
        #print("Keys in 'client_local_test_log':", client_local_model_log_keys)
        client_local_model_log_data = {}
        for key in client_local_model_log_keys:
            try:
                client_local_model_log_data[key] = f['client_local_model_log'][key][:]
            except KeyError:
                pass
                
        client_local_test_log_keys = list(f['client_local_test_log'].keys())
        # Iterate over the keys and extract the corresponding data
        client_local_test_log_data = {}
        for key in client_local_test_log_keys:
            client_local_test_log_data[key] = f['client_local_test_log'][key][:]
    
        client_gradnorm_log_keys = list(f['gradient_norm_lists_by_client'].keys())
        # Iterate over the keys and extract the corresponding data
        client_gradnorm_log_data = {}
        for key in client_gradnorm_log_keys:
            client_gradnorm_log_data[key] = f['gradient_norm_lists_by_client'][key][:]
    
        if "FEDAVG" in directory_str:
            global_dec_log = f['global_dec_log'][()]

if False:
    print(client_local_model_log_keys)
    print()
    for i in range(NUM_CLIENTS):
        print(client_local_model_log_data[client_local_model_log_keys[i]].shape)
    print()
    print(global_dec_log.shape)

#from sklearn.neighbors import KernelDensity
#from sklearn.metrics import mutual_info_score
#import numpy as np
#from sklearn.decomposition import PCA
#from sklearn.feature_selection import mutual_info_regression

if False:
    # Find the longest array based on shape[0]
    max_length = max(client_local_model_log_data[key].shape[0] for key in client_local_model_log_keys)
    # Generate the indices for equally spaced sampling from global_dec_log
    indices = np.linspace(0, len(global_dec_log) - 1, max_length).astype(int)
    # Sample global_dec_log to match the length of the longest client array
    sampled_global_dec_log = global_dec_log[indices]

    # So now I need to link each set of 20 consecutive models to each update. Eg 1-20-->F1, 21-40-->F2, ... 140+-->F7 (might not get to F7!)
    ## So it should iterative through consecutive models
    
    # THIS PART DOESNT WORK: MI ISNT DEFINED / WORK WELL BETWEEN TRAINING DATA AND WEIGHTS, APPARENTLY...
    ## Maybe I could just ignore that and flatten the model weights and it would be fine? Not sure...
    cli_mi_local = {i: [] for i in range(NUM_CLIENTS)}
    cli_mi_global = {i: [] for i in range(NUM_CLIENTS)}
    for i in range(NUM_CLIENTS):
        cli_i_model_log = client_local_model_log_data[client_local_model_log_keys[i]]
        if cli_i_model_log.shape[0]<2:
            # Client didnt train (eg it was probably a testset client)
            print(f"Cli {i} skipped for being too short!")
        else:
            for model_idx, model in enumerate(cli_i_model_log):
                # Models and dataset may not be aligned
                ## Eg some may be saved in order of 0, 10, 11, 12, 13, 1, ... whereas the other may be saved as 0, 1, 2, 3, ...
    
                if model_idx > local_round_threshold * num_usuable_updates:
                    input_index = 7
                else:
                    input_index = model_idx // local_round_threshold
                # Access the corresponding input for this chunk
                X_train = client_F_dict[i][input_index]  # TODO: THIS DOES NOT SUPPORT 2 BATCH PFA RIGHT NOW!
                #print(f"Model idx: {model_idx}; Update (input) index: {input_index}'")
    
                # COMPARE EACH LOCAL MODEL TO THE TRAINING DATA TO GENERATE A MUTUAL INFORMATION CURVE
                # Step 1: Dimensionality reduction of training data using PCA
                # Reduce X_train to 2 dimensions to match model weights shape
                pca = PCA(n_components=2)
                X_train_PCA = pca.fit_transform(X_train).flatten()
                
                # Step 2: Compute mutual information between reduced training data and local model weights
                cli_mi_local[i].append(mutual_info_regression(X_train_PCA, model.flatten()))
                
                # COMPARE EACH GLOBAL MODEL (from the adjusted list) TO THE TRAINING DATA
                #if model_idx%20==0:
                cli_mi_global.append(mutual_info_regression(X_train_PCA, sampled_global_dec_log[model_idx].flatten()))

if False:
    # Helper function to compute entropy using KDE
    def kde_entropy(X, bandwidth=1.0):
        kde = KernelDensity(bandwidth=bandwidth)
        kde.fit(X)
        log_density = kde.score_samples(X)
        return -np.mean(log_density)
    
    # Helper function to compute mutual information using KDE
    def kde_mutual_information(X, Y, bandwidth=1.0):
        # Joint entropy H(X,Y)
        XY = np.hstack([X, Y])
        joint_entropy = kde_entropy(XY, bandwidth)
        
        # Marginal entropies H(X) and H(Y)
        entropy_X = kde_entropy(X, bandwidth)
        entropy_Y = kde_entropy(Y, bandwidth)
        
        # MI = H(X) + H(Y) - H(X,Y)
        mutual_info = entropy_X + entropy_Y - joint_entropy
        return mutual_info
    
    # Dummy training data (64 examples, 1202 features)
    X_train = np.random.rand(64, 1202)
    
    # Dummy local model weights (64 examples, 2 features)
    client_local_model_weights = np.random.rand(64, 2)
    
    # Dummy global model weights (64 examples, 2 features)
    global_model_weights = np.random.rand(64, 2)
    
    # Step 1: Reduce the training data dimensionality using PCA to match model weights (optional, depending on your scenario)
    # Alternatively, you can use summary statistics, but let's use PCA here for KDE method
    pca = PCA(n_components=2)
    X_train_reduced = pca.fit_transform(X_train)
    
    # Step 2: Compute KDE-based mutual information between reduced training data and local model weights
    mi_local_kde = kde_mutual_information(X_train_reduced, client_local_model_weights)
    
    # Step 3: Compute KDE-based mutual information between reduced training data and global model weights
    mi_global_kde = kde_mutual_information(X_train_reduced, global_model_weights)
    
    # Output mutual information scores
    print("KDE-based Mutual Information between X_train and local model weights:", mi_local_kde)
    print("KDE-based Mutual Information between X_train and global model weights:", mi_global_kde)


## Model Inversion Attack

In [ ]:
if False:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    
    # Function to perform model inversion attack
    def model_inversion_attack(model, target_output, input_shape, num_iterations=1000, learning_rate=1e-2):
        # Initialize input (with random noise)
        inverted_input = torch.randn(input_shape, requires_grad=True)
        optimizer = optim.Adam([inverted_input], lr=learning_rate)
        
        loss_fn = nn.MSELoss()
        
        for i in range(num_iterations):
            optimizer.zero_grad()
            output = model(inverted_input)
            loss = loss_fn(output, target_output)
            loss.backward()
            optimizer.step()
        
        return inverted_input.detach()
    
    # Example usage
    client_models = [ClientModel() for _ in range(14)]  # Placeholder for client models
    target_model = client_models[0]  # Assume you are attacking the first client
    target_output = torch.tensor([0.1, 0.9])  # Target output we want to reproduce
    
    # Perform inversion attack to get reconstructed input
    inverted_input = model_inversion_attack(target_model, target_output, input_shape=(1, 28, 28))  # Example input shape (e.g. for images)
    print("Reconstructed input:", inverted_input)


In [ ]:
# Gradient Descent Model Inversion Attack on a Linear Regression Model
def model_inversion_attack(weights, target_output, input_shape, num_iterations=1000, learning_rate=0.01):
    # Initialize a random input (e.g., random noise)
    inverted_input = np.random.randn(*input_shape)
    
    for i in range(num_iterations):
        # Compute the output from the current inverted input
        output = np.dot(inverted_input, weights)
        
        # Compute the loss (Mean Squared Error between target and current output)
        loss = np.mean((output - target_output) ** 2)
        
        # Compute gradient of the loss with respect to the input
        grad = 2 * np.dot((output - target_output), weights.T) / len(target_output)
        
        # Update the inverted input by moving in the direction of the negative gradient
        inverted_input -= learning_rate * grad
        
        if i % 100 == 0:  # Print loss every 100 iterations
            print(f"Iteration {i}, Loss: {loss}")
    
    return inverted_input

# Example usage:
np.random.seed(42)

# Assume a simple linear regression model (2 features, 1 target)
weights = np.array([[2.0, -1.0]]).T  # A 2x1 weight matrix

# Target output we want to reproduce
target_output = np.array([0.5])

# Perform the model inversion attack to find an input that reproduces the target output
inverted_input = model_inversion_attack(weights, target_output, input_shape=(1, 2), num_iterations=1000, learning_rate=0.01)

print("Reconstructed input:", inverted_input)
